# Übung 8 - JobPosts Similarity Implementation of Topic Modeling LDA algorithm
In this notebook the LDA (Latent Dirichlet Allocation) algorithmen will be implemented via gensim libary and visualized via pyLDAvis. 
For more information about libaries see down

-------------

#### 1. Import Libaries
For more Information see:  
- gensim: https://radimrehurek.com/gensim/
- pyLDAvis: https://pyldavis.readthedocs.io/en/latest/readme.html

In [1]:
!pip install --upgrade gensim #install gensim libary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 76.9 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
!pip install pyldavis #install pyLDAVIS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=0cfdbe7b79d9b7ead8494de905f9dbe0276f4c23eabf352c1f8dcf651dd5dc9a
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyldavis


In [3]:
!pip install openpyxl #install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora
from nltk.corpus import stopwords
import spacy
import pyLDAvis
from gensim.models import LdaModel
import pyLDAvis.gensim_models as gensimvis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


-------------

#### 2. Load Data Set

In [7]:
df = pd.read_excel("./data_scientist_united_states_job_postings_jobspikr.xlsx")
df

,crawl_timestamp,url,job_title,category,company_name,city,state,country,inferred_city,inferred_state,...,job_description,job_type,salary_offered,job_board,geo,cursor,contact_email,contact_phone_number,uniq_id,html_job_description
0,2019-02-06 06:26:22,https://www.indeed.com/viewjob?jk=fd83355c2b23...,Enterprise Data Scientist I,Accounting/Finance,Farmers Insurance Group,Woodland Hills,CA,Usa,Woodland hills,California,...,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549432819114777,NaN,NaN,3b6c6acfcba6135a31c83bd7ea493b18,NaN
1,2019-02-06 06:33:41,https://www.dice.com/jobs/detail/Data-Scientis...,Data Scientist,NaN,Luxoft USA Inc,Middletown,NJ,Usa,Middletown,New jersey,...,We have an immediate opening for a Sharp Data ...,Undefined,NaN,dice,usa,1549432819122106,NaN,NaN,741727428839ae7ada852eebef29b0fe,NaN
2,2019-02-06 06:33:35,https://www.dice.com/jobs/detail/Data-Scientis...,Data Scientist,NaN,Cincinnati Bell Technology Solutions,New York,NY,Usa,New york,New york,...,Candidates should have the following backgroun...,Full Time,NaN,dice,usa,1549432819236156,NaN,NaN,cdc9ef9a1de327ccdc19cc0d07dbbb37,NaN
3,2019-02-06 06:33:42,https://www.indeed.com/viewjob?jk=841edd86ead2...,"Data Scientist, Aladdin Wealth Tech, Associate...",Accounting/Finance,BlackRock,New York,NY 10055 (Midtown area),Usa,New york,New york,...,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549432819259473,NaN,NaN,1c8541cd2c2c924f9391c7d3f526f64e,NaN
4,2019-02-06 06:48:23,https://job-openings.monster.com/senior-data-s...,Senior Data Scientist,biotech,CyberCoders,Charlotte,NC,Usa,Charlotte,North carolina,...,We are seeking an extraordinary Data Scientist...,Full Time,NaN,monster,usa,1549436429015957,NaN,NaN,445652a560a5441060857853cf267470,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2019-09-29 02:55:15,https://stackoverflow.com/jobs/271829,Data Scientist,Data Scientist,Upstart,Columbus,NaN,US,Columbus,Wisconsin,...,Join Upstart and help build the future of cred...,Full Time,$100k - 135k | Equity,Stackoverflow,NaN,1569916870107016,NaN,NaN,e9f2ca58c7abcd4553a4a0be9602312e,<h2>About this job</h2>_x000D_\n<p><span>Compe...
9996,2019-10-01 02:50:33,https://stackoverflow.com/jobs/283895,"Data Scientist, (Staff or Principal)",Data Scientist,Realtor.com,Santa Clara,NaN,US,Santa clara,New mexico,...,"Data Scientist, (Staff or Principal) at realto...",Full Time,NaN,Stackoverflow,NaN,1569916870211396,NaN,NaN,efe05f2364ee3df943e952bc68ed95a7,<h2>About this job</h2>_x000D_\n<p><span>Locat...
9997,2019-09-26 02:53:10,https://stackoverflow.com/jobs/293326,"Siri - Embedded Data Scientist, Data Organization","Data Scientist, Embedded Developer",Apple Inc.,Pittsburgh,NaN,US,Pittsburgh,Pennsylvania,...,Job Summary: Would you like to play a part in ...,Full Time,NaN,Stackoverflow,NaN,1569916870956579,NaN,NaN,b980054dbb6d3fe1c6a421a237d1495f,NaN
9998,2019-09-27 02:41:31,https://stackoverflow.com/jobs/283895,"Data Scientist, (Staff or Principal)",Data Scientist,Realtor.com,Santa Clara,NaN,US,Santa clara,New mexico,...,"Data Scientist, (Staff or Principal) at realto...",Full Time,NaN,Stackoverflow,NaN,1569916871226964,NaN,NaN,4b86da9d2b24b87ec3c815d9072ad616,<h2>About this job</h2>_x000D_\n<p><span>Locat...


#### 2.1 Load different numbers of samples and remove Nan-values


In [8]:
c = df["job_description"].iloc[0:1000] #Change here the sample size (1000) for better testing
corpus = list(c.dropna())
#corpus

Im diesem Ansatz wird nur das Feature "job_description" experimentiert, da man versuchen möchte ganze Beschreibungen Themen zuzuordnen. Da man im letzten Notebook um Features behandelt wurden, die keien Sätze besitzen, versucht man mit dem LDA Ansatz, die ähnliche Texte in selbe Themen zuzuordnen.

#### 2.2 Preprocess the input documents

In [ ]:
#A small function to preproceses documents
def clean_text(documents):
  nlp = spacy.load("en_core_web_sm")
  new_corpus = []
  for document in documents:
    filler = ""
    doc = nlp(document)
    for token in doc:
      if token.is_stop:
        continue 
        
      if token.is_alpha:
        filler += " " +  token.lemma_

    new_corpus += [filler[1:len(filler)]]

  return new_corpus

In [ ]:
new_corpus = clean_text(corpus)

In [ ]:
#new_corpus

#### 2.3 Tokenize documents
For more Information see:
- gensim simple_preprocess(): https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html

In [ ]:
#A small function to tokenize the text
def preprocess(documents):
  new_documents = []
  for document in documents:
    tokens = gensim.utils.simple_preprocess(document) #convert text into token like spacy
    new_documents += [tokens]

  return new_documents


In [ ]:
new_corpus2 = preprocess(new_corpus)

In [ ]:
#new_corpus2[2][0:20]

#### 2.4 Transform words into vectors (Bag-of-words approach)
Assign unique id's to each word.
For more information see:
- gensim corpora.Dictionary(): https://radimrehurek.com/gensim/corpora/dictionary.html
- transform into vectors: https://radimrehurek.com/gensim/auto_examples/core/run_corpora_and_vector_spaces.html

In [ ]:
dictionary = corpora.Dictionary(new_corpus2)
def transform(documents):
  corpus = []
  for document in documents:
    vector = dictionary.doc2bow(document) #transform each text to vectors
    corpus += [vector]

  return corpus

In [ ]:
new_corpus3 = transform(new_corpus2)

In [ ]:
#dictionary.token2id

In [ ]:
#new_corpus3[0][0:20] #returns (id of the word , frequency of the word)

-------------------------

#### 3. Initalize LDA Model and train it
For more information see:
- gensim LDA model: https://radimrehurek.com/gensim/models/ldamodel.html
- training model: https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html

In [ ]:
model = LdaModel(
    corpus=new_corpus3,
    id2word=dictionary,
    alpha='auto',
    eta='auto',
    iterations=400,
    num_topics=10,
    eval_every=None)

#### 4. Visualize results
for more information see:
- pyLDAvis prepare(): https://pyldavis.readthedocs.io/en/latest/modules/API.html

In [ ]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, new_corpus3, dictionary)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.164330 -0.003804       1        1  58.365802
6      0.164289  0.015169       2        1  16.638998
8      0.163138 -0.042304       3        1  13.273737
2      0.138049 -0.016073       4        1   6.509414
7      0.118166  0.067140       5        1   5.038018
4     -0.053715 -0.027142       6        1   0.087135
1     -0.114423 -0.014314       7        1   0.035092
0     -0.179212  0.009155       8        1   0.034967
9     -0.222216 -0.000832       9        1   0.009609
5     -0.178405  0.013005      10        1   0.007228, topic_info=            Term          Freq         Total Category  logprob  loglift
16           and  21679.000000  21679.000000  Default  30.0000  30.0000
192           to  13987.000000  13987.000000  Default  29.0000  29.0000
145         pron  11183.000000  11183.000000  Default  28.0000  28.0000
121           of   8943.000000   8943.000000  Default  27.0000  27.0000
189          the  10448.000000  10448.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
126           or      0.017999   4154.363682  Topic10  -7.4303  -2.8145
109      machine      0.016370   1739.364380  Topic10  -7.5252  -2.0387
376  information      0.015365    813.796092  Topic10  -7.5885  -1.3425
27         build      0.014994   1054.826281  Topic10  -7.6130  -1.6264
26            be      0.015649   7527.523015  Topic10  -7.5702  -3.5487

[770 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
6362      1  0.238111  absorb
6362      5  0.714334  absorb
1266      1  0.631464  access
1266      2  0.164730  access
1266      3  0.076874  access
...     ...       ...     ...
224       5  0.033663    year
224       6  0.000748    year
224       7  0.000748    year
5679      3  0.828554    yolo
5680      3  0.779803   yourâ

[1496 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 7, 9, 3, 8, 5, 2, 1, 10, 6])

**Ergebnis**
Wie man in der Visualisierung sehen kann, hat der LDA Algorithmus eigentlich nur 5 Themen ausgewählt, da die restlichen fünf Themen (7-10) Ausreißer darstelllen. Was man auch in der Visualiserierung sehen kann, ist, das die zugeordneten Wörter zu den jeweiligen Themen, eher Stoppwörter sind bzw. am meisten zufällige Wörter, die dem Kontext der Beschreibung keinen Sinn ergeben anzutreffen. Wenn man spezifische Wörter rausfiltert und vielleich spezifisches Preprocessing verwendet kann man den Topic Modeling LDA Ansatz verwenden um (hier für komplexere Dokumente, die Sätze haben) Themen zuzuweisen, die man mit herkömlichen Ähnlichkeitsmaße nicht machen könnte.